<a href="https://colab.research.google.com/github/arshad62/learn-agentic-ai/blob/main/06b_crew_ai/05_augmented_llms/crewai_crews_advance_concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import os

from google.colab import userdata

os.environ['GEMINI_API_KEY'] =  userdata.get('GEMINI_API_KEY')
os.environ['OPENAI_API_KEY'] =  userdata.get('OPENAI_API_KEY')

In [2]:
!pip install -Uq crewai crewai-tools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 42.5 MB

In [ ]:
!pip install --upgrade chromadb
!pip install --upgrade tokenizers

  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.5.23
    Uninstalling chromadb-0.5.23:
      Successfully uninstalled chromadb-0.5.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.127 requires chromadb<0.6.0,>=0.5.10, but you have chromadb 0.6.3 which is incompatible.


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from crewai.flow.flow import Flow, start, listen


class MyFlow(Flow):

  @start()
  def function1(self):
    print("step1")

  @listen(function1)
  def function2(self):
    print("step2")

obj = MyFlow()
obj.kickoff()

 Flow started with ID: 779ba370-05a8-4720-ac63-9c7ea7617808
step1
step2


In [7]:
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
)

llm1.call("What is the capital of France?")

'The capital of France is Paris.\n'

In [8]:
google_embedder = {
    "provider": "google",
    "config": {
         "model": "models/text-embedding-004",
         "api_key": userdata.get('GEMINI_API_KEY'),
         }
}

In [9]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Create a knowledge source
content = "Users name is Arshad. He is 62 years old and lives in Doha, Qatar. He is working as Telecome Consultant at MoD"
string_source = StringKnowledgeSource(
    content=content,
)


# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    memory=True,
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source], # Enable knowledge by adding the sources here. You can also add more sources to the sources list.
    embedder=google_embedder

)

result = crew.kickoff(inputs={"question": "What city does Arshad live in and how old is he?"})

# Agent: About User
## Task: Answer the following questions about the user: What city does Arshad live in and how old is he?


# Agent: About User
## Final Answer: 
Arshad lives in Doha, Qatar, and he is 62 years old.




In [10]:
import os
os.environ['CREWAI_STORAGE_DIR']='/my_crew1'

In [11]:
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
)

In [12]:
from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
# from crewai.memory.storage import LTMSQLiteStorage, RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage

from typing import List, Optional

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    memory = True,
    # Long-term memory for persistent storage across sessions
    long_term_memory = LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="./my_crew2/long_term_memory_storage1.db"
        )
    ),
    # Short-term memory for current context using RAG
    short_term_memory = ShortTermMemory(
        storage = RAGStorage(
                embedder_config=google_embedder,
                type="short_term",
                path="./my_crew2/short_term1/"
            )
        ),

    # Entity memory for tracking key information about entities
    entity_memory = EntityMemory(
        storage=RAGStorage(
            embedder_config=google_embedder,
            type="short_term",
            path="./my_crew2/entity1/"
        )
    ),
    verbose=True,
)

# Integrating Mem0 for Enhanced User Memory


In [13]:
import os
from crewai import Crew, Process
from mem0 import MemoryClient

# Set environment variables for Mem0
os.environ["MEM0_API_KEY"] = userdata.get('MEM0_API_KEY')

# Step 1: Record preferences based on past conversation or user input
client = MemoryClient()
# messages = [
#     {"role": "user", "content": "Hi there! I'm planning a vacation and could use some advice."},
#     {"role": "assistant", "content": "Hello! I'd be happy to help with your vacation planning. What kind of destination do you prefer?"},
#     {"role": "user", "content": "I am more of a beach person than a mountain person."},
#     {"role": "assistant", "content": "That's interesting. Do you like hotels or Airbnb?"},
#     {"role": "user", "content": "I like Airbnb more."},
# ]
# client.add(messages, user_id="john")

# Step 2: Create a Crew with User Memory

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    verbose=True,
    memory=True,
    memory_config={
        "provider": "mem0",
        "config": {"user_id": "john"},
    },
)

crew.kickoff(inputs={"question": "What is your favorite vacation destination?"})

# Agent: About User
## Task: Answer the following questions about the user: What is your favorite vacation destination?


# Agent: About User
## Final Answer: 
Since you do not have a favorite vacation destination, I am unable to provide an answer.




/usr/local/lib/python3.11/dist-packages/mem0/client/main.py:32: DeprecationWarning: Using default output format 'v1.0' is deprecated and will be removed in version 0.1.70. Please use output_format='v1.1' for enhanced memory details. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


CrewOutput(raw='Since you do not have a favorite vacation destination, I am unable to provide an answer.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Answer the following questions about the user: What is your favorite vacation destination?', name=None, expected_output='An answer to the question.', summary='Answer the following questions about the user: What is your...', raw='Since you do not have a favorite vacation destination, I am unable to provide an answer.', pydantic=None, json_dict=None, agent='About User', output_format=<OutputFormat.RAW: 'raw'>)], token_usage=UsageMetrics(total_tokens=270, prompt_tokens=212, cached_prompt_tokens=0, completion_tokens=58, successful_requests=1))

# Tools

In [ ]:
#!pip install --upgrade transformers
#!pip install --upgrade tokenizers
#!pip install -Uq 'crewai[tools]'

  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.127 requires chromadb<0.6.0,>=0.5.10, but you have chromadb 0.6.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.49.0 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


In [ ]:
#!pip install --upgrade chromadb

  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
Using cached chromadb-0.6.3-py3-none-any.whl (611 kB)
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.5.23
    Uninstalling chromadb-0.5.23:
      Successfully uninstalled chromadb-0.5.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.127 requires chromadb<0.6.0,>=0.5.10, but you have chromadb 0.6.3 which is incompatible.


In [1]:
import os

from google.colab import userdata

os.environ['GEMINI_API_KEY'] =  userdata.get('GEMINI_API_KEY')
#os.environ['OPENAI_API_KEY'] =  userdata.get('OPENAI_API_KEY')

In [2]:
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [3]:
!pip install -Uq crewai crewai-tools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 47.2 MB

In [4]:
embedder = {
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": userdata.get('GEMINI_API_KEY'),
        }
    }

In [5]:
!pip install -Uq langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [ ]:
!pip install crewai crewai-tools

In [7]:
from crewai import LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm=LLM(model="gemini/gemini-1.5-flash")

In [9]:
!pip install --upgrade crewai-tools

In [12]:
from crewai_tools import tools

print(tools)

<module 'crewai_tools.tools' from '/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/__init__.py'>


In [15]:
from crewai import Agent

# Initialize an agent with the Gemini model
agent = Agent(
    role='Research Analyst',
    goal='Provide insights on recent AI developments',
    backstory='An AI assistant specializing in AI research.',
    llm=llm
)


In [13]:
#from crewai_tools import
class GeminiWebsiteSearchTool():
    def __init__(self, llm):
        self.llm = llm

    def run(self, query: str):
        response = self.llm.generate(query)
        return response

In [16]:
# Initialize the custom tool
gemini_search_tool = GeminiWebsiteSearchTool(llm=llm)

# Assign the tool to the agent
agent.tools = [gemini_search_tool]


In [19]:
from crewai import Agent, Task, Crew

In [21]:
from crewai import Agent, Task, Crew

# Initialize an agent with the Gemini model
agent = Agent(
    role='Research Analyst',
    goal='Provide insights on recent AI developments',
    backstory='An AI assistant specializing in AI research.',
    llm=llm
)

# Initialize the custom tool
gemini_search_tool = GeminiWebsiteSearchTool(llm=llm)

# Assign the tool to the agent
agent.tools = [gemini_search_tool]

# Define a task for the agent
task = Task(
    description="Research the latest advancements in AI technology.",
    expected_output="A summary of the latest advancements in AI technology.",
    agent=agent
)

# Create a crew with the agent and task
crew = Crew(agents=[agent], tasks=[task])

# Agent performs a web search using the crew
result = crew.kickoff()  # Call kickoff on the Crew object
print(result)

AttributeError: 'GeminiWebsiteSearchTool' object has no attribute 'name'

In [25]:
#!pip install openai
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY') # serper.dev API key
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
# Example with WebsiteSearchTool
web_rag_tool = WebsiteSearchTool(llm=llm)

# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True,
    llm=llm
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True,
    llm=llm
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the AI industry, based on the research analyst’s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)

# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True,  # Enable planning feature
    planning_llm=llm
)

# Execute tasks
crew.kickoff()

 
[2025-03-06 10:06:01][INFO]: Planning the crew execution
# Agent: Market Research Analyst
## Task: Research the latest trends in the AI industry and provide a summary.Step 1: The Market Research Analyst will utilize the 'Search the internet with Serper' tool.  The search query will be crafted to focus on the most recent advancements in AI.  Specific keywords such as "latest AI trends", "breakthroughs in artificial intelligence", "cutting-edge AI technologies", and  "emerging AI applications" should be incorporated to ensure a comprehensive search. The search type should be set to 'news' to prioritize recent developments.

Step 2:  The analyst will carefully review the top 10 results returned by Serper.  The focus will be on identifying recurring themes and significant developments across multiple sources to ensure objectivity and avoid bias from individual articles.  Specific attention should be paid to identifying trends that are truly impactful and transformative, not just incremen

CrewOutput(raw="# The AI Revolution: Three Trends Shaping Our Future\n\nThe world of artificial intelligence is moving at breakneck speed, transforming industries and reshaping our daily lives.  From creative tools that generate stunning art to algorithms that revolutionize healthcare, AI is no longer a futuristic fantasy; it's the present and the future. Three key trends are pushing this revolution forward, promising both incredible opportunities and critical challenges that we must address.\n\n\nOne of the most exciting developments is the rise of generative AI.  Large language models (LLMs) like ChatGPT and image generators like DALL-E 2 are capable of creating incredibly realistic and original content – everything from poems and code to marketing copy and medical diagnoses.  These models are not just toys; they're being integrated into business workflows, streamlining tasks, and offering personalized experiences for customers.  While this technology brings the potential for increas

In [26]:
# prompt: import markdown for jupyter display

from IPython.display import Markdown

def display_markdown_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    display(Markdown(content))

# Example usage: Assuming 'blog-posts/new_post.md' is the file generated by your code
display_markdown_file('blog-posts/new_post.md')


# The AI Revolution: Three Trends Shaping Our Future

The world of artificial intelligence is moving at breakneck speed, transforming industries and reshaping our daily lives.  From creative tools that generate stunning art to algorithms that revolutionize healthcare, AI is no longer a futuristic fantasy; it's the present and the future. Three key trends are pushing this revolution forward, promising both incredible opportunities and critical challenges that we must address.


One of the most exciting developments is the rise of generative AI.  Large language models (LLMs) like ChatGPT and image generators like DALL-E 2 are capable of creating incredibly realistic and original content – everything from poems and code to marketing copy and medical diagnoses.  These models are not just toys; they're being integrated into business workflows, streamlining tasks, and offering personalized experiences for customers.  While this technology brings the potential for increased efficiency and new creative avenues, it also raises important questions about job displacement and the responsible use of this powerful tool.


Hand-in-hand with the advancement of AI capabilities comes a growing awareness of the ethical implications.  Bias in algorithms, data privacy concerns, and the potential for misuse are serious challenges that demand our attention.  Governments and organizations are grappling with the need for robust regulations and ethical guidelines to ensure that AI development is responsible and benefits humanity as a whole.  Transparency and accountability are crucial, and the ongoing dialogue between technologists, policymakers, and the public is essential for navigating the ethical complexities of this rapidly evolving field.


Finally, the widespread adoption of AI across various sectors is undeniable.  From healthcare, where AI is assisting in diagnosis and drug discovery, to finance, where it's enhancing fraud detection and risk management, AI is proving its value across numerous industries. This adoption leads to increased efficiency and productivity, but also necessitates investment in education and retraining to equip the workforce for the changing landscape.  The long-term impact will be a transformation of how we work and live, making it crucial to address challenges like data security and privacy to ensure that this revolution benefits everyone.
```

## Creating your own Tools

In [27]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

class MyToolInput(BaseModel):
    """Input schema for MyCustomTool."""
    student_name: str = Field(..., description="Student name")
    student_roll_no : int = Field(..., description="student id")

class PiaicStudentCard(BaseTool):
    name: str = "Piaic student card generator"
    description: str = "this function will create Piaic student card"
    args_schema: Type[BaseModel] = MyToolInput

    def _run(self, student_name: str, student_roll_no: int ) -> str:
        # Your tool's logic here
        return f"""PIAIC student card
student name: {student_name}
student roll no: {student_roll_no}
Pakistan zindabd!
        """

In [28]:
from crewai.tools import tool

@tool("PIAIC fee update")
def my_tool(roll_no: int) -> dict | str:
    """this function search piaic student fee updates, it will required roll no of PIAIC student"""
    #database

    data = {100:'paid',
         200:'unpaid'}


    status = data.get(roll_no)

    if status:
      return {"status": status}
    else:
      return "student not found"





In [29]:
from crewai import LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm1=LLM(model="gemini/gemini-1.5-flash")

In [30]:
from crewai import Agent, Task, Crew

card = PiaicStudentCard()


piaic_manager = Agent(
    role="PIAIC manager",
    goal = "Manage all quries regarding PIAIC and you will use only relevant tools for student query",
    backstory="""You are a master at understanding people and their preferences.""",
    tools=[card, my_tool],
    verbose=True,
    llm=llm1
)

piaic_card_creator = Task(
    description="you will be responsible for all PIAIC relevant operations, student query '{query}' you must be know how to answer his question based on final context",
    expected_output="final query answer only",
    agent=piaic_manager
)

crew = Crew(
    agents=[piaic_manager],
    tasks=[piaic_card_creator],
    verbose=True
)

result = crew.kickoff(inputs={
    "query":"I'm PIAIC student my name is Muhammad Qasim and my roll number is 100, can you create my student card."
})

print(result)

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student my name is Muhammad Qasim and my roll number is 100, can you create my student card.' you must be know how to answer his question based on final context


# Agent: PIAIC manager
## Using tool: Piaic student card generator
## Tool Input: 
"{\"student_name\": \"Muhammad Qasim\", \"student_roll_no\": 100}"
## Tool Output: 
PIAIC student card
student name: Muhammad Qasim
student roll no: 100
Pakistan zindabd!
        


# Agent: PIAIC manager
## Final Answer: 
PIAIC student card
student name: Muhammad Qasim
student roll no: 100
Pakistan zindabd!
```


PIAIC student card
student name: Muhammad Qasim
student roll no: 100
Pakistan zindabd!
```


In [31]:
result = crew.kickoff(inputs={
    "query":"I'm PIAIC student  my roll number is 100, can you share my fee updates?."
})

print(result)

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student  my roll number is 100, can you share my fee updates?.' you must be know how to answer his question based on final context


# Agent: PIAIC manager
## Using tool: PIAIC fee update
## Tool Input: 
"{\"roll_no\": 100}"
## Tool Output: 
{'status': 'paid'}


# Agent: PIAIC manager
## Final Answer: 
Your fee is paid.
```


Your fee is paid.
```


In [32]:
result = crew.kickoff(inputs={
    "query":"I'm PIAIC student  my roll number is 200, can you share my fee updates?."
})

print(result)

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student  my roll number is 200, can you share my fee updates?.' you must be know how to answer his question based on final context


# Agent: PIAIC manager
## Using tool: PIAIC fee update
## Tool Input: 
"{\"roll_no\": 200}"
## Tool Output: 
{'status': 'unpaid'}


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Piaic student card generator
Tool Arguments: {'student_name': {'description': 'Student name', 'type': 'str'}, 'student_roll_no': {'description': 'student id', 'type': 'int'}}
Tool Description: this function will create Piaic student card
Tool Name: PIAIC fee update
Tool Arguments: {'roll_no': {'description': None, 'type': 'int'}}
Tool Description: this function search piaic student fee updates, it will required roll no of PIAIC student

IMPORTANT: Use the following format in your response:

```
Thoug

# Process

In [33]:
import os
from crewai import Agent, Task, Crew, Process

# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)



# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts2')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

# Define your agents
researcher = Agent(
    role="Researcher",
    goal="Conduct thorough research and analysis on AI and AI agents",
    backstory="You're an expert researcher, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently researching for a new client.",
    allow_delegation=False,
    tools=[search_tool,web_rag_tool],
    llm=llm1
)

writer = Agent(
    role="Senior Writer",
    goal="Create compelling content about AI and AI agents",
    backstory="You're a senior writer, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently writing content for a new client.",
    allow_delegation=False,
    tools=[docs_tool,file_tool],
    llm=llm1
)

# Define your task
task = Task(
    description="Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with their paragraphs and your notes.",
    expected_output="5 bullet points, each with a paragraph and accompanying notes.",
)

# Define the manager agent
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
    llm=llm1
)

# Instantiate your crew with a custom manager
crew = Crew(
    agents=[researcher, writer],
    tasks=[task],
    manager_agent=manager,
    function_calling_llm=llm1,
    process=Process.hierarchical,
    verbose=True

)

# Start the crew's work
result = crew.kickoff()
from IPython.display import Markdown

Markdown(result.raw)

# Agent: Project Manager
## Task: Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with their paragraphs and your notes.


# Agent: Project Manager
## Thought: ```tool_code
Thought: I need to delegate the task of generating article ideas to the Senior Writer and then ask the Researcher to help refine them.  I'll then delegate the paragraph writing to the Senior Writer.
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": {\"description\": \"Generate 5 interesting article ideas related to [insert any relevant field here, if needed, otherwise leave blank].  Ideas should be unique and have potential for a full-length article.  Each idea should be concise, one sentence per idea.\", \"type\": \"str\"}, \"context\": {\"description\": \"The goal is to create a list of 5 engaging article ideas that demonstrate the potential for a full-l

* The Unexpected Benefits of Learning a New Language:  Explore the cognitive, social, and career advantages of multilingualism, challenging common misconceptions and highlighting surprising benefits that extend beyond communication.  Notes:  Focus on cognitive research (improved memory, multitasking), social benefits (increased empathy, cross-cultural understanding), and career prospects.

* The Psychology of Procrastination: Delve into the psychological mechanisms behind procrastination, offering practical strategies and coping mechanisms to help readers overcome this common productivity barrier. Notes: Include different types of procrastination, cognitive biases involved, and evidence-based techniques for overcoming it.

* The Power of Micro-Habits:  Examine the transformative potential of tiny, consistent actions, demonstrating how accumulating small changes can lead to significant improvements in various aspects of life. Notes: Provide specific examples (exercise, reading, learning) and emphasize the importance of consistency over intensity.

* Redefining Success:  Challenge conventional definitions of success, exploring alternative perspectives and promoting a more holistic and personally fulfilling approach to goal setting and achievement. Notes:  Include interviews with people who have unconventional definitions of success and discuss the importance of personal values.

* The Future of Work:  Analyze current trends in the workplace, examining the impact of technology, globalization, and demographic shifts on the future of employment and the skills needed to thrive in a rapidly evolving job market.  Notes: Include data on future job markets and projections, potential job displacement due to AI, and the need for adaptable skills and lifelong learning.
```

# Advance Tools

In [34]:
from pydantic import BaseModel, Field

class TeacherAssignmentOutPut(BaseModel):
  score :int  = Field(..., description="student assignment score")
  feedback : str = Field(..., description="teacher feedback")
  question : str = Field(..., description="question")
  answer : str = Field(..., description="python code for markdown format")

In [35]:
from crewai import Agent
from crewai_tools import CodeInterpreterTool
from crewai import Task, Crew, Process

teacher = Agent(
    role="AI teacher",
    goal="you are Agentic AI teacher you have to check python assignment code submission.",
    backstory="you have to check student python code submission.",
    tools=[CodeInterpreterTool()],
    llm=llm1
)

assignment_check = Task(
    description="you are Agentic AI teacher you have to check python assignment code submission. question: '''{qestion}''' student solution : '''{solution}''' ",
    expected_output="final code is runing, assign number between 1-10",
    output_pydantic=TeacherAssignmentOutPut,
    agent=teacher # Added the missing agent here
)


crew = Crew(
    agents=[teacher],
    tasks=[assignment_check],
    process=Process.sequential,
    verbose=True,
)

question = "create function add two numbers"

solution = """
def add_two_number(num1 : int , num2 : int):
  return num1 + num2

"""

result = crew.kickoff(inputs={
    "qestion": question,
    "solution" : solution
})

print(result)

# Agent: AI teacher
## Task: you are Agentic AI teacher you have to check python assignment code submission. question: '''create function add two numbers''' student solution : '''
def add_two_number(num1 : int , num2 : int):
  return num1 + num2

''' 


# Agent: AI teacher
## Thought: Thought:I need to run the student's code to check its correctness and then provide a score and feedback.
## Using tool: Code Interpreter
## Tool Input: 
"{\"code\": \"def add_two_number(num1 : int , num2 : int):\\n  return num1 + num2\\n\\nprint(add_two_number(2,3))\\nprint(add_two_number(-1,1))\\nprint(add_two_number(0,0))\", \"libraries_used\": []}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory')).
 Tool Code Interpreter accepts these inputs: Tool Name: Code Interpreter
Tool Arguments: {'code': {'description': 'Python3 code used to be interpreted i

In [ ]:
result.to_dict()

{'score': 9,
 'feedback': 'The code is correct and defines a function that adds two numbers. The type hints are a good addition. A small improvement could be adding a docstring to explain what the function does.',
 'question': 'create function add two numbers',
 'answer': 'def add_two_number(num1 : int , num2 : int):\n  return num1 + num2'}

In [ ]:
for k,v in result.to_dict().items():
  print(str("*"*10) + str(k) + str("*"*10))
  display(v)

**********score**********


9

**********feedback**********


"The function correctly adds two numbers.  Consider adding docstrings for better readability and to explain the function's purpose."

**********question**********


'create function add two numbers'

**********answer**********


'def add_two_number(num1, num2):\n  return num1 + num2\n'